In [2]:
import snowflake.connector
import pandas as pd
from datasets import Dataset, ClassLabel
from dotenv import load_dotenv

In [3]:
load_dotenv()

True

In [4]:
conn = snowflake.connector.connect(
    user=os.environ["SF_USER"],
    password=os.environ["SF_PWD"],
    account=os.environ["SF_ACCOUNT"],
    database=os.environ["SF_DB"],
    schema=os.environ["SF_SCHEMA"]
)

In [5]:
# query the data from Snowflake and create a Pandas dataframe
query = 'SELECT * FROM intent_dataset;'
df = pd.read_sql(query, conn)

/var/folders/4n/j6_091hn32x0t8shq_lb0pxc0000gn/T/ipykernel_87235/2729437409.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [16]:
df

,TEXT,CATEGORY,SOURCE
0,I am still waiting on my card?,card_arrival,train
1,What can I do if my card still hasn't arrived ...,card_arrival,train
2,I have been waiting over a week. Is the card s...,card_arrival,train
3,Can I track my card while it is in the process...,card_arrival,train
4,"How do I know if I will get my card, or if it ...",card_arrival,train
...,...,...,...
13078,"If i'm not in the UK, can I still get a card?",country_support,test
13079,How many countries do you support?,country_support,test
13080,What countries do you do business in?,country_support,test
13081,What are the countries you operate in.,country_support,test


In [89]:
from transformers import DistilBertForSequenceClassification, DistilBertTokenizerFast, Trainer, TrainingArguments
from datasets import Features, Value

In [90]:
# get the list of unique labels in the 'category' column
label_list = df['CATEGORY'].unique().tolist()

# instantiate a ClassLabel object with the number of classes and the names of the labels
num_classes = len(label_list)
# create Hugging Face dataset
features = Features({"TEXT": Value("string"), 
                     "CATEGORY": ClassLabel(num_classes=num_classes, names=label_list),
                     "SOURCE": Value("string"),})
dataset = Dataset.from_pandas(df, features=features)


In [91]:
dataset = dataset.rename_column("CATEGORY", "label")

In [92]:
# split dataset into train and test sets based on the value of "SOURCE"
train_dataset = dataset.filter(lambda example: example["SOURCE"] == "train")
test_dataset = dataset.filter(lambda example: example["SOURCE"] == "test")

Filter:   0%|          | 0/13083 [00:00<?, ? examples/s]

Filter:   0%|          | 0/13083 [00:00<?, ? examples/s]

In [93]:
# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples['TEXT'], padding="max_length", truncation=True)

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/10003 [00:00<?, ? examples/s]

Map:   0%|          | 0/3080 [00:00<?, ? examples/s]

# Train Model
This does not work and needs to be fixed

In [94]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments

# load the pre-trained BERT tokenizer
tokenizer = AutoTokenizer.from_pretrained("philschmid/BERT-Banking77")

# load the pre-trained small BERT model for sequence classification
model = AutoModelForSequenceClassification.from_pretrained("philschmid/BERT-Banking77", num_labels=num_classes)

In [95]:
# set up the training arguments
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    evaluation_strategy='epoch',     # evaluate model after every epoch
    learning_rate=2e-5,              # learning rate
    per_device_train_batch_size=16,  # batch size for training
    per_device_eval_batch_size=64,   # batch size for evaluation
    num_train_epochs=3,              # number of training epochs
    weight_decay=0.01,               # weight decay
    push_to_hub=False,               # whether to upload the model checkpoint to the Hub
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

# create the Trainer object
trainer = Trainer(
    model=model,                         # the instantiated model to be trained
    args=training_args,                  # training arguments
    train_dataset=train_dataset,         # the training dataset
    eval_dataset=test_dataset            # the evaluation dataset
)

In [96]:
# start the training
trainer.train()

  0%|          | 0/1878 [00:00<?, ?it/s]

{'loss': 7.4833, 'learning_rate': 1.9893503727369543e-05, 'epoch': 0.02}
{'loss': 5.4309, 'learning_rate': 1.9787007454739085e-05, 'epoch': 0.03}
{'loss': 4.7968, 'learning_rate': 1.9680511182108627e-05, 'epoch': 0.05}
{'loss': 4.2412, 'learning_rate': 1.957401490947817e-05, 'epoch': 0.06}
{'loss': 3.7701, 'learning_rate': 1.946751863684771e-05, 'epoch': 0.08}
{'loss': 3.4793, 'learning_rate': 1.9361022364217256e-05, 'epoch': 0.1}
{'loss': 3.1575, 'learning_rate': 1.9254526091586797e-05, 'epoch': 0.11}
{'loss': 2.6047, 'learning_rate': 1.914802981895634e-05, 'epoch': 0.13}
{'loss': 2.3512, 'learning_rate': 1.904153354632588e-05, 'epoch': 0.14}
{'loss': 2.0893, 'learning_rate': 1.8935037273695422e-05, 'epoch': 0.16}
{'loss': 1.8338, 'learning_rate': 1.8828541001064964e-05, 'epoch': 0.18}
{'loss': 1.8665, 'learning_rate': 1.8722044728434506e-05, 'epoch': 0.19}
{'loss': 1.5325, 'learning_rate': 1.8615548455804048e-05, 'epoch': 0.21}
{'loss': 1.5875, 'learning_rate': 1.850905218317359e-05,

  0%|          | 0/49 [00:00<?, ?it/s]

{'eval_loss': 0.4730672538280487, 'eval_runtime': 1717.0914, 'eval_samples_per_second': 1.794, 'eval_steps_per_second': 0.029, 'epoch': 1.0}
{'loss': 0.2064, 'learning_rate': 1.329073482428115e-05, 'epoch': 1.01}
{'loss': 0.1509, 'learning_rate': 1.3184238551650694e-05, 'epoch': 1.02}
{'loss': 0.2388, 'learning_rate': 1.3077742279020236e-05, 'epoch': 1.04}
{'loss': 0.1106, 'learning_rate': 1.2971246006389777e-05, 'epoch': 1.05}
{'loss': 0.177, 'learning_rate': 1.2864749733759319e-05, 'epoch': 1.07}
{'loss': 0.0997, 'learning_rate': 1.2758253461128862e-05, 'epoch': 1.09}
{'loss': 0.1512, 'learning_rate': 1.2651757188498404e-05, 'epoch': 1.1}
{'loss': 0.1889, 'learning_rate': 1.2545260915867944e-05, 'epoch': 1.12}
{'loss': 0.1728, 'learning_rate': 1.243876464323749e-05, 'epoch': 1.13}
{'loss': 0.1883, 'learning_rate': 1.233226837060703e-05, 'epoch': 1.15}
{'loss': 0.1458, 'learning_rate': 1.2225772097976571e-05, 'epoch': 1.17}
{'loss': 0.1995, 'learning_rate': 1.2119275825346113e-05, 'ep